In [1]:
import pandas as pd
import uproot
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import crystalball
from scipy.optimize import curve_fit,Bounds
from scipy.integrate import simps
import zfit_physics as zphysics
import zfit
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.pyplot as plt
import mplhep
import uncertainties 
from uncertainties import ufloat
import tensorflow as tf

plt.style.use('belle2')
#%jsroot on


%run /afs/desy.de/user/j/jipatel/MT/MC_fit/Files_After_Acceptance/Signal_Background_thesis_final/Function.ipynb

2024-09-03 12:52:54.921915: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:52:54.953439: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-03 12:52:54.954321: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 12:52:56.461149: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/__init__.py:63: UserWarning: TensorFlow warnings are by default suppressed by zfit. In order to show them, set the environment variable ZFIT_DISABLE_TF_WARNINGS=0. In order to suppress the TensorFlow warnings AND this warnin

## 200

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 200
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"],  'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"], 'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"],  'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"],  'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"], 'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)


    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_200.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_200.csv'")


Running iteration 1/300...
200
1023


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7082                 223.221  -     17   +     18       False
background_yield_0.7082             999.301  -     32   +     34       False
frac_Mbc_bkg1_0.7082               0.379441  -  0.048   +  0.048       False
frac_deltaE_bkg1_0.7082          -0.0319549  -   0.21   +    0.2       False
True
Iteration 1 - Signal Yield: 223.22068512393736 ± 17.735685928307976
Running iteration 2/300...
200
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1494                 231.755  -     18   +     18       False
background_yield_0.1494             991.263  -     33   +     33       False
frac_Mbc_bkg1_0.1494               0.364792  -  0.048   +  0.048       False
frac_deltaE_bkg1_0.1494         -0.00838743 

## 150

In [3]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 150
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_150.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_150.csv'")


Running iteration 1/300...
150
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3769                 174.131  -     16   +     16       False
background_yield_0.3769             998.518  -     32   +     34       False
frac_Mbc_bkg1_0.3769                 0.3795  -  0.048   +  0.047       False
frac_deltaE_bkg1_0.3769          -0.0231294  -   0.21   +   0.21       False
True
Iteration 1 - Signal Yield: 174.13087658605707 ± 16.10377841779262
Running iteration 2/300...
150
1023
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.485                 178.418  -     16   +     16       False
background_yield_0.485             994.247  -     32   +     34       False
frac_Mbc_bkg1_0.485               0.372169  -  0.049   +  0.047       False
frac_deltaE_bk

## 100

In [4]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 100
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_100.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_100.csv'")


Running iteration 1/300...
100
1023
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.499                 128.265  -     14   +     15       False
background_yield_0.499             994.932  -     33   +     33       False
frac_Mbc_bkg1_0.499               0.372109  -  0.048   +  0.047       False
frac_deltaE_bkg1_0.499         -0.00473065  -   0.21   +   0.21       False
True
Iteration 1 - Signal Yield: 128.26498319595134 ± 14.36624162309383
Running iteration 2/300...
100
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0253                 127.861  -     14   +     15       False
background_yield_0.0253             995.452  -     33   +     33       False
frac_Mbc_bkg1_0.0253               0.373126  -  0.048   +  0.047       False
frac_deltaE_bkg

## 75

In [5]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 75
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_75.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_75.csv'")


Running iteration 1/300...
75
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.2881                 100.357  -     13   +     14       False
background_yield_0.2881              997.59  -     32   +     33       False
frac_Mbc_bkg1_0.2881               0.377367  -  0.047   +  0.048       False
frac_deltaE_bkg1_0.2881           0.0185481  -   0.21   +   0.21       False
True
Iteration 1 - Signal Yield: 100.35695763829388 ± 13.246380016093045
Running iteration 2/300...
75
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3828                 98.6441  -     13   +     14       False
background_yield_0.3828             999.376  -     32   +     33       False
frac_Mbc_bkg1_0.3828               0.380701  -  0.047   +  0.047       False
frac_delta

## 50

In [6]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 50
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])

# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_50.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_50.csv'")


Running iteration 1/300...
50
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.1135                 74.0232  -     12   +     12       False
background_yield_0.1135             998.463  -     32   +     34       False
frac_Mbc_bkg1_0.1135               0.381386  -  0.048   +  0.046       False
frac_deltaE_bkg1_0.1135           0.0142577  -    0.2   +   0.21       False
True
Iteration 1 - Signal Yield: 74.0232031545039 ± 12.027412100867789
Running iteration 2/300...
50
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7707                 68.8603  -     12   +     12       False
background_yield_0.7707             1003.47  -     32   +     34       False
frac_Mbc_bkg1_0.7707               0.389795  -  0.047   +  0.047       False
frac_deltaE_

## 25

In [7]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 25
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_25.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_25.csv'")


Running iteration 1/300...
25
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.7865                 47.8184  -     10   +     11       False
background_yield_0.7865             1000.38  -     33   +     33       False
frac_Mbc_bkg1_0.7865               0.381611  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.7865           0.0308859  -    0.2   +   0.21       False
True
Iteration 1 - Signal Yield: 47.8183810842295 ± 10.578532474098472
Running iteration 2/300...
25
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3336                 46.7639  -     10   +     11       False
background_yield_0.3336             1001.44  -     33   +     33       False
frac_Mbc_bkg1_0.3336                0.38468  -  0.047   +  0.047       False
frac_deltaE_

## 10

In [9]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 10
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_10.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_10.csv'")


Running iteration 1/300...
10
1023
name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.623                 32.3975  -    9.3   +     10       False
background_yield_0.623             1000.19  -     32   +     33       False
frac_Mbc_bkg1_0.623               0.383354  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.623           0.0404754  -    0.2   +    0.2       False
True
Iteration 1 - Signal Yield: 32.397540426282745 ± 9.73400143596244
Running iteration 2/300...
10
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.8111                 28.6612  -    9.2   +     10       False
background_yield_0.8111             1004.36  -     32   +     33       False
frac_Mbc_bkg1_0.8111               0.390028  -  0.047   +  0.047       False
frac_deltaE_bkg1_0

## 5

In [11]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 5
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_5.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_5.csv'")


Running iteration 1/300...
5
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.6401                 27.6793  -      9   +    9.8       False
background_yield_0.6401             1000.47  -     32   +     33       False
frac_Mbc_bkg1_0.6401                0.38294  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.6401           0.0374235  -    0.2   +    0.2       False
True
Iteration 1 - Signal Yield: 27.679295715843093 ± 9.387872377599736
Running iteration 2/300...
5
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.5012                 25.0157  -    8.7   +    9.8       False
background_yield_0.5012             1002.32  -     32   +     34       False
frac_Mbc_bkg1_0.5012               0.388625  -  0.048   +  0.046       False
frac_deltaE_b

## 2

In [12]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 2
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_2.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_2.csv'")


Running iteration 1/300...
2
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.4259                 25.5694  -    8.7   +    9.5       False
background_yield_0.4259             999.489  -     32   +     33       False
frac_Mbc_bkg1_0.4259               0.381738  -  0.047   +  0.047       False
frac_deltaE_bkg1_0.4259           0.0253544  -    0.2   +   0.21       False
True
Iteration 1 - Signal Yield: 25.56940033026545 ± 9.142986986666198
Running iteration 2/300...
2
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.0824                 24.4873  -    8.7   +    9.5       False
background_yield_0.0824             1000.58  -     32   +     33       False
frac_Mbc_bkg1_0.0824               0.383165  -  0.047   +  0.047       False
frac_deltaE_bk

## 0

In [2]:
# Number of iterations
num_iterations = 300

# List to store the signal yields and their errors from each iteration
signal_yields_and_errors = []

for i in range(num_iterations):
    print(f"Running iteration {i + 1}/{num_iterations}...")

    # Process the Monte Carlo data
    mc_mu_signal, mc_mu_only_signal, mc_mu_scf, mc_mu_bkg, mc_mu_signal_weight, mc_mu_only_signal_weight, mc_mu_scf_weight, mc_mu_bkg_weight = process_MC_data_combined(
        file_signal_mu_B0pip,file_bkg_mu_B0pip, 'mu', '511', 'pip', 'mumu', 3, 'rec511', 'weights', 0
    )

    # Fit models for signal, SCF, and background
    Mbc_only_signal_cb_fixed = fit_fixed_without_weight('Mbc', 'mu', 'signal', mc_mu_only_signal["Mbc"], 'cb', 0)
    deltaE_only_signal_cb_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'signal', mc_mu_only_signal["deltaE"],  'cb_gauss', 0)

    Mbc_scf_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'scf', mc_mu_scf["Mbc"], 'argus_gauss', 0)
    deltaE_scf_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'scf', mc_mu_scf["deltaE"], 'poly_gauss', 0)

    Mbc_bkg_argus_gauss_fixed = fit_fixed_without_weight('Mbc', 'mu', 'bkg', mc_mu_bkg["Mbc"], 'argus_gauss', 0)
    deltaE_bkg_poly_gauss_fixed = fit_fixed_without_weight('deltaE', 'mu', 'bkg', mc_mu_bkg["deltaE"],  'poly_gauss', 0)

    # Combine signal and background data
    Signal_Mbc = mc_mu_only_signal["Mbc"]
    Signal_deltaE = mc_mu_only_signal["deltaE"]
    Background_Mbc = pd.concat([mc_mu_scf["Mbc"], mc_mu_bkg["Mbc"]])
    Background_deltaE = pd.concat([mc_mu_scf["deltaE"], mc_mu_bkg["deltaE"]])

    Signal_total = pd.concat([Signal_Mbc, Signal_deltaE], axis=1)
    Background_total = pd.concat([Background_Mbc, Background_deltaE], axis=1)

    MC_signal=len(Signal_total)
    MC_bkg=len(Background_total)

    print(MC_signal)
    print(MC_bkg)
    # Fit the extended model and get the result
    Mbc_signal_pdf, deltaE_signal_pdf, Mbc_background_pdf, deltaE_background_pdf, signal_extended, background_extended, total_model, result = fit_extended_model_scf_without_weight(
        Mbc_only_signal_cb_fixed, Mbc_scf_argus_gauss_fixed, deltaE_only_signal_cb_gauss_fixed, deltaE_scf_poly_gauss_fixed,
        Mbc_bkg_argus_gauss_fixed, deltaE_bkg_poly_gauss_fixed,
        pd.concat([Signal_total, Background_total]), 0.5, 0.5, 100, 100
    )
    result.errors(name='errors')
    print(result.params)
    print(result.valid)

    # Retrieve and store the signal yield and its error
    signal_yield = result.params[signal_extended.get_yield()]['value']
    signal_yield_errors = result.params[signal_extended.get_yield()]['errors']
    signal_yield_error = (signal_yield_errors['upper'] - signal_yield_errors['lower']) / 2

    # Store the signal yield and error as a tuple in the list
    signal_yields_and_errors.append((MC_signal,MC_bkg,signal_yield, signal_yield_error))

    print(f"Iteration {i + 1} - Signal Yield: {signal_yield} ± {signal_yield_error}")

# Convert the list to a DataFrame with two columns: 'Signal_Yield' and 'Signal_Yield_Error'
signal_yields_df = pd.DataFrame(signal_yields_and_errors, columns=['MC_signal','MC_bkg','Signal_Yield', 'Signal_Yield_Error'])


# Save the DataFrame to a CSV file
signal_yields_df.to_csv('B0pip_q2reg3_decay_mumu_signal_yields_and_errors_0.csv', index=False)

print("Signal yields and errors saved to 'B0pip_q2reg3_decay_mumu_signal_yields_and_errors_0.csv'")


Running iteration 1/300...
0
1023


/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/core/basepdf.py:585: UserWarning: As `copy` is not yet properly implemented, this may fails (for ProductPDF for example?). Thiswill be fixed in the future.
  warnings.warn(
/afs/desy.de/user/j/jipatel/.local/lib/python3.8/site-packages/zfit/minimizers/fitresult.py:1340: FutureWarning: 'minuit_minos' will be changed as the default errors method to a custom implementationwith the same functionality. If you want to make sure that 'minuit_minos' will be used in the future, add it explicitly as in `errors(method='minuit_minos')`
  warnings.warn(


name                      value  (rounded)               errors    at limit
----------------------  ------------------  -------------------  ----------
signal_yield_0.884                 22.4072  -    8.6   +    9.4       False
background_yield_0.884             1000.62  -     32   +     33       False
frac_Mbc_bkg1_0.884               0.383336  -  0.046   +  0.047       False
frac_deltaE_bkg1_0.884            0.034555  -    0.2   +    0.2       False
True
Iteration 1 - Signal Yield: 22.40723871287726 ± 8.96550548842095
Running iteration 2/300...
0
1023
name                       value  (rounded)               errors    at limit
-----------------------  ------------------  -------------------  ----------
signal_yield_0.3284                 22.4072  -    8.6   +    9.4       False
background_yield_0.3284             1000.62  -     32   +     33       False
frac_Mbc_bkg1_0.3284               0.383336  -  0.046   +  0.047       False
frac_deltaE_bkg1_0.3284            0.034555  -    0.2  